In [114]:
# buy when the candle closes without a bottom wick or the bwick is 20% of the body or less
# sell on the first red czndle

# if the wick bigger than the body sell ----maybe----
# if the upper or lower wick is long i wont enter 
############################################################

# heiken ashi is only good for detecting trend reversal 
# the higher the timeframe the better
# daily are very much preferable

# detect the first green candle after a series of a red ones
# organize them by the number of red candles


# look for longest running streak of red candles 
# and keep an eye on it because once the green candle appears 
# it will go like crazy

In [115]:
import pandas as pd
import pandas_ta as ta
import os
from IPython.display import display, HTML


In [116]:
# if (ha.iloc[-1]['HA_close'] > ha.iloc[-1]['HA_open']):
#     # find the start of streak

def heiken(ha):
    ha=ta.ha(data['Open'],data['High'],data['Low'],data['Close'])
    # find all green candles
    ha['greens'] = (ha['HA_open'] < ha['HA_close']) 
    # ha.tail(19)

    # if last candle is green 
    # loop ha in reverse (from last to first)
    # break at the first false(wivh means a red candle)
    # print the index of red candle found
    # now you have a streak start with red, ends with green
    if (ha.iloc[-1]['HA_close'] > ha.iloc[-1]['HA_open']):
        for i in reversed(range(len(ha))):
            if ha['greens'][i] == False:
                # gcc is green candles count
                # by substracting 1 we get only greens count
                gcc = len(ha.iloc[i:]) - 1
                # print(f"{ha.index[i]} , candle count is {gcc} (last green candle and first red candle included)")
                # candles.append([symbol, gcc, ha.index[i]])

                # find how many red candles
                for j in reversed(range(i-1)):
                    if ha['greens'][j] == True:
                        # print(j)
                        # first green candle and last red candle included
                        # do real count is rcc = rcc-2
                        rcc = len(ha.iloc[j:i])

                        # if there is only one red candle skip and continue looking
                        # 3 because 2 green candle are always accounted for
                        # so 2 green candle and 1 red cabdle 
                        # if rcc == 3:
                        #     continue

                        break

                
                candles.append([symbol, gcc, ha.index[i], rcc])


                break
            
    # dd = pd.DataFrame({'symbols': symbol, 'candle_count': cc, 'Time': ha.index[i]})



    

In [117]:
# list_of=[]
timeframe='1d'
candles=[]
# dirr = f"C:/Users/Grant/Desktop/work_git/work/data/2022-01-28_12/{timeframe}"
outdir=f"{os.getcwd()}/data/{timeframe}"

for filename in os.listdir(outdir):
    # symbol = (filename.replace(f"_{tf}.csv","")).replace("_","")
    symbol = filename.replace(f"_{timeframe}.csv","")
    data = pd.read_csv(f"{outdir}/{filename}", index_col='Time', parse_dates=['Time'])
    # print(symbol)

    # speed up the process
    # before it was 50s now its 9s
    data = data[-66:]
    
    heiken(data)

    # looks like its the same thing; puting the table variable inside the loop or
    # outside it; actualy there is a slight increase in performance
    # if i put table var outstide the loop
    
    # table = pd.DataFrame(rows, columns=["symbols", "candle_count","Time"])

table = pd.DataFrame(candles, columns=["symbols", "g_candles_count","Time_red_candle", "r_candles_count"])

table['link'] = table['symbols'].apply(lambda x: f'<a href=\
    "https://www.tradingview.com/chart/UOC7kIDx/?symbol=BINANCE%3A{x.replace("/","")}">{x}</a>')

display(HTML(table.to_html(escape=False)))

# table

,symbols,g_candles_count,Time_red_candle,r_candles_count,link
0,ACHUSDT,2,2022-02-18,2,ACHUSDT
1,ANCUSDT,3,2022-02-17,2,ANCUSDT
2,API3USDT,8,2022-02-12,2,API3USDT
3,ATMUSDT,1,2022-02-19,4,ATMUSDT
4,AUTOUSDT,1,2022-02-19,3,AUTOUSDT
5,BETAUSDT,2,2022-02-18,2,BETAUSDT
6,FARMUSDT,1,2022-02-19,9,FARMUSDT
7,IDEXUSDT,1,2022-02-19,7,IDEXUSDT
8,INJUSDT,2,2022-02-18,6,INJUSDT
9,KMDUSDT,1,2022-02-19,4,KMDUSDT
